In [26]:
from faker import Faker
import json
import sqlite3
import random
import secrets
from datetime import date, timedelta

fake = Faker()

In [27]:
# generate crypto id
def generate_crypto_id():
    return secrets.token_hex(36)

# fetch the orders from the database
def fetch_orders():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM orders')
    orders = c.fetchall()
    conn.close()
    return orders

# select the orders id with status completed from the order list
def select_orders(status1, status2, orders):
    completed_orders = []
    for order in orders:
        if order[2] == status1 or order[2] == status2:
            completed_orders.append(order[0])
    return completed_orders

# select order id with status completed for shipping
def select_orders_shipping(status1, orders):
    processing_orders = []
    for order in orders:
        if order[2] == status1:
            processing_orders.append(order[0])
    return processing_orders

# from list of order id get the address
def lookup_address(order_id, orders):
    for order in orders:
        if order[0] == order_id:
            return order[7]

# from the order id look up the total_price
def lookup_total_price(order_id, orders):
    for order in orders:
        if order[0] == order_id:
            return order[3]
        
# gacha the payment methods
def gacha_payment_method():
    payment_methods = [
        'cash',
        'bank transfer',
        'check',
        'credit card',
        'electronic wallet',
    ]
    weight = [
        0.4,
        0.1,
        0.4,
        0.05,
        0.05
    ]
    return random.choices(payment_methods, weight, k=1)[0]

# generate payment data in the future
def generate_payment_date():
    return fake.date_between(start_date=date(2025,5,1), end_date=date(2025,12,31)).strftime('%Y-%m-%d')

In [28]:
orders = fetch_orders()
completed_orders = select_orders('completed', 'processing', orders)
payments = []

# create payment
for i in range(len(completed_orders)):
    payments.append(
        {
            'id': generate_crypto_id(),
            'order_id': completed_orders[i],
            'date': generate_payment_date(),
            'payment_method': gacha_payment_method(),
            'total_price': lookup_total_price(completed_orders[i], orders)
        }
    )

payments_json = json.dumps(payments, indent=4)

# save the as json file
with open('json_data/payments.json', 'w') as f:
    f.write(payments_json)

In [29]:
orders = fetch_orders()
processing_orders = select_orders_shipping('processing', orders)
shipping = []   

# create shipping
for i in range(len(processing_orders)):
    ship_date = fake.date_between(start_date=date(2025,5,1), end_date=date(2025,12,31))
    random_time_delta = random.randint(1, 6)
    delivery_date = ship_date + timedelta(days=random_time_delta)
    shipping.append(
        {
            'id': generate_crypto_id(),
            'order_id': processing_orders[i],
            'address': lookup_address(processing_orders[i], orders),
            'shipping_date': ship_date.strftime('%Y-%m-%d'),
            'delivery_date': delivery_date.strftime('%Y-%m-%d'),
        }
    )
    
shipping_json = json.dumps(shipping, indent=4)

# save the as json file
with open('json_data/shippings.json', 'w') as f:
    f.write(shipping_json)

In [30]:
# insert the data into the database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS payments (
        id TEXT PRIMARY KEY NOT NULL,
        order_id TEXT NOT NULL,
        date DATE NOT NULL,
        payment_method TEXT NOT NULL,
        total_price REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders (id)
        )
    '''
)

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS shippings (
        id TEXT PRIMARY KEY NOT NULL,
        order_id TEXT NOT NULL,
        address TEXT NOT NULL,
        shipping_date DATE NOT NULL,
        delivery_date DATE NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders (id)
        )
    '''
)

for payment in payments:
    c.execute(
        '''
        INSERT INTO payments (id, order_id, date, payment_method, total_price)
        VALUES (?, ?, ?, ?, ?)
        ''',
        (payment['id'], payment['order_id'], payment['date'], payment['payment_method'], payment['total_price'])
    )
    
for ship in shipping:
    c.execute(
        '''
        INSERT INTO shippings (id, order_id, address, shipping_date, delivery_date)
        VALUES (?, ?, ?, ?, ?)
        ''',
        (ship['id'], ship['order_id'], ship['address'], ship['shipping_date'], ship['delivery_date'])
    )
    
conn.commit()
conn.close()